# Pipeline pour la préparation des données en inférence

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Charger le tokenizer commun
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Charger les modèles pré-entraînés pour le domaine général et pour le COVID
# Remplacez "path_to_general_model" et "path_to_covid_model" par le chemin ou l'identifiant de vos modèles
model_general = BertForSequenceClassification.from_pretrained("path_to_general_model")
model_covid = BertForSequenceClassification.from_pretrained("path_to_covid_model")

def predict_fake_news(text, domain="general"):
    """
    Prédit si un article est Fake ou Real en fonction du domaine spécifié par l'utilisateur.

    Args:
        text (str): Le texte de l'article.
        domain (str): Le domaine souhaité ("general" ou "covid").

    Returns:
        tuple: Une chaîne indiquant "Fake" ou "Real", et le domaine utilisé.
    """
    # Préparation du texte
    encoding = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Sélection du modèle en fonction du domaine choisi
    if domain.lower() == "covid":
        model_to_use = model_covid
    else:
        model_to_use = model_general

    model_to_use.eval()
    with torch.no_grad():
        outputs = model_to_use(**encoding)

    # Prédiction : 1 pour Fake, 0 pour Real
    prediction = torch.argmax(outputs.logits, dim=1).item()
    result = "Fake" if prediction == 1 else "Real"

    return result, domain

# Exemple d'utilisation
if __name__ == "__main__":
    text_exemple = "Les nouvelles récentes sur la pandémie COVID-19 indiquent une évolution surprenante de la situation."
    result, chosen_domain = predict_fake_news(text_exemple, domain="covid")
    print(f"Domaine sélectionné : {chosen_domain} | Prédiction : {result}")
